# Connecting to Hardware

**SUMMARY**: *In this lab, you'll learn how to setup and connect to your ChipWhisperer hardware. We'll also cover how to build firmware for your target microcontroller, how to capture power traces, and how to communicate with target devices.*

*All the API calls we'll be using here are documented on the [ChipWhisperer readthedocs page](https://chipwhisperer.readthedocs.io/en/latest/index.html#api), so feel free to open it in another tab and follow along there as well.*

**LEARNING OUTCOMES:**
* Setting up your ChipWhisperer Hardware
* Using the ChipWhisperer Python API to connect to your hardware
* Communication with the target
* Capturing a power trace


## Physical Setup

### ChipWhisperer-Nano

The ChipWhisperer-Nano is a single-board device. It includes the capture hardware, along with a built in STM32F0 target. To use this target, simply plug in the device, then go get a drink to reward yourself:

<img src="img/cwnano.jpg" alt="CW-Nano Plugged In" width=400>

For the tutorials, you should use `PLATFORM='CWNANO'`.

## Connecting to ChipWhisperer

Now that your hardware is all setup, we can now learn how to connect to it. We can connect to the ChipWhisperer with:

In [1]:
import chipwhisperer as cw
scope = cw.scope()

By default, ChipWhisperer will try to autodetect the type of device your're running (CWLite/CW1200 or CWNano), see API documentation for manually specifying the scope type. If you have multiple ChipWhisperer devices connected, you'll need to specify the serial number of the device you want to connect to:

```python
scope = cw.scope(sn='<some long string of numbers>')
```

For more information, see the API section on readthedocs.

Connecting to the target device is simple as well:

In [2]:
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted

We'll only be discussing the default target type, which is SimpleSerial. Other targets, like the CW305, will be covered in hardware specific demos. 

Some sane default settings can be set using:

In [3]:
scope.default_setup()

which from its [documentation](https://chipwhisperer.readthedocs.io/en/latest/api.html#chipwhisperer.capture.scopes.OpenADC.OpenADC.default_setup) you can see does the following for the CWLite/CW1200:

* Sets the scope gain to 45dB
* Sets the scope to capture 5000 samples
* Sets the scope offset to 0 (aka it will begin capturing as soon as it is triggered)
* Sets the scope trigger to rising edge
* Outputs a 7.37MHz clock to the target on HS2
* Clocks the scope ADC at 4\*7.37MHz. Note that this is *synchronous* to the target clock on HS2
* Assigns GPIO1 as serial RX
* Assigns GPIO2 as serial TX

And that's it! Your ChipWhisperer is now setup and ready to attack a target. 

**NOTE: You'll need to disconnect the scope/target before connecting again, like you would in another notebook. This can be done with `scope.dis()` and `target.dis()`**

## Building and Uploading Firmware

The next step in attacking a target is to get some firmware built and uploaded onto it. Most firmware for most ChipWhisperer targets can be built using our build system, provided you have the correct compiler installed (see https://chipwhisperer.readthedocs.io/en/latest/prerequisites.html#prerequisites for info about compilers).

Firmware must be built on the command line. Luckily, thanks to Jupyter, we can run a command within a notebook as follows:

In [4]:
%%bash
cd ../hardware/victims/firmware/simpleserial-base/
make PLATFORM='CWNANO' CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
    simpleserial-base.c ...Done!
.
Compiling:
    .././simpleserial/simpleserial.c ...Done!
.
Compiling:
    .././hal/stm32f0_nano/stm32f0_hal_nano.c ...Done!
.
Compiling:
    .././hal/stm32f0/stm32f0_hal_lowlevel.c ...Done!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.

You should see a big list of `PLATFORM`s to build for. We left the `PLATFORM` blank in the command above, so the build system instead printed a list of supported platforms. Fill in your platform, rerun the build command, and your firmware should be successfully built.

Continuing on, there's two possible ways to upload firmware to your target:

1. ChipWhisperer has built in support for XMEGA, STM32F*, AVR, and SAM4S bootloaders. These can be used as follows

In [5]:
#cw.program_target(scope, cw.programmers.XMEGAProgrammer, "path/to/firmware.hex")
cw.program_target(scope, cw.programmers.STM32FProgrammer, "../hardware/victims/firmware/simpleserial-base/simpleserial-base-CWNANO.hex")
#cw.program_target(scope, cw.programmers.AVRProgrammer, "path/to/firmware.hex")
#cw.program_target(scope, cw.programmers.SAM4SProgrammer, "path/to/firmware.hex")

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4655 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4655 bytes


2. For other targets, you'll need to use an external programmer or a debugger to flash the firmware onto the target. 

Whatever your case, upload the firmware we built earlier to the target device. Next we'll be learning how to capture power traces and communicate with the target.

## Communication with the Target

Communication with targets, which is done through the `SimpleSerial target` object we got earlier, is grouped into two categories:

1. Raw serial via `target.read()`, `target.write()`, `target.flush()`, etc. 

1. SimpleSerial commands via `target.simpleserial_read()`, `target.simpleserial_write()`, `target.simpleserial_wait_ack()`, etc.

The firmware we uploaded uses the simpleserial protocol (https://wiki.newae.com/SimpleSerial), so we'll start off with simpleserial. Later, we'll use the raw serial commands to send the same messages.

If you check the simpleserial-base firmware (`simpleserial-base.c`) you'll find that for the simpleserial `'p'` command, the target will echo back the command. Let's try that out now:

In [6]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
target.simpleserial_write('p', msg)

Let's check if we got a response:

In [7]:
print(target.simpleserial_read('r', 16))

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


It also has a `'k'` command. Try sending that now:

In [8]:
target.simpleserial_write('k', msg)

That command doesn't return anything to us, but it should ack and give us an error return:

In [9]:
print(target.simpleserial_wait_ack()) #should return 0

0


Simpleserial messages generally take the form:

```python
command_character + ascii_encoded_bytes + '\n'
```

For our first command, `command_character='p'` and `ascii_encoded_bytes="00"*16` (keep in mind this isn't a binary `0x00`, it's ASCII `"00"`, which has a binary value of `0x3030`). Try resending the `'p'` command from earlier using `target.write()`:

In [10]:
target.write('p' + "00"*16+'\n') #fill in the rest here

A simple `target.read()` will return all the characters that have been sent back from the target so far. Let's see what the device returned to us:

In [11]:
recv_msg = ""

In [12]:
recv_msg += target.read() #you might have to run this block a few times to get the full message
print(recv_msg)

r00000000000000000000000000000000
z00



The simpleserial commands are usually sufficient for taking to simpleserial firmware, but you'll need the raw serial commands for some of the other labs.

## SimpleSerial 2

As of ChipWhisperer 5.4, a new target communication protocol, SimpleSerial 2 is available. It has a number of advantages over the old SimpleSerial protocol:

1. Data is unencoded binary (except for byte stuffing) instead of ACII encoded, meaning far more data can be sent per packet (just under double)
1. It's got a command and subcommand field that get passed to the target callback functions, meaning more can be done per packet. For example, the simpleserial-aes firmware has been modified to allow plaintext, key, and/or mask to be set with a single packet.
1. It's got a length field, meaning the same target commands can take different length packets depending on the situation. This, for example, allows both plaintext and key (or only one) to be send with the same command in simpleserial-aes.
1. It's got an 8-bit CRC (0xA6) for data integrety.
1. Frames are consistant overhead byte stuffed (COBS). This makes it easy to reset communication (done simply by sending two 0x00 bytes) and helps catch malformed length bytes.

It is, however, a more complicated protocol. You cause use it by compiling firmware with `SS_VER=SS_VER_2_0` and using `cw.targets.SimpleSerial2`. It can be used by setting `SS_VER='SS_VER_2_0'` when setting `PLATFORM` and `SCOPETYPE`.

## Capturing Traces

Now that the target's programmed and we know how to communicate with it, let's start recording some power traces! To capture a trace:

1. Arm the ChipWhisperer with `scope.arm()`. It will begin capturing as soon as it is triggered (which in our case is a rising edge on `gpio4`).
1. `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return will tell you whether or not the scope timed out. It does not return the captured scope data.
1. You can read back the captured power trace with `scope.get_last_trace()`.

`simpleserial_base` will trigger the ChipWhisperer when we send the `'p'` command. Try capturing a trace now:

In [15]:
scope.arm()
target.simpleserial_write('p', msg)

In [16]:
## fill in the rest...
scope.capture()
scope.get_last_trace()

array([-0.02734375,  0.13671875, -0.05859375, ..., -0.08984375,
       -0.11328125,  0.25390625])

ChipWhisperer also has a `capture_trace()` convience function that:

1. Optionally sends the `'k'` command
1. Arms the scope
1. Sends the `'p'` command
1. Captures the trace
1. Reads the return `'r'` message
1. Returns a `Trace` class that groups the trace data, `'p'` message, the `'r'` message, and the `'k'` message

It isn't always the best option to use, but it's usually sufficient for most simpleserial applications

## Conclusion

And that's it! You should be all ready to continue on to step 3 of the lab.

We've glossed over some stuff here, so consult the [API documentation](https://chipwhisperer.readthedocs.io/en/latest/index.html#api) or ask on our [forums](https://forum.newae.com) if you get stuck.

As a final step, we should disconnect from the hardware so it doesn't stay "in use" by this notebook.

In [17]:
scope.dis()
target.dis()

Congratulations! You are finished with Step 3.

Move on to Step 4 to continue the lab.